In [36]:
#!/usr/bin/python3

import os
import sys
import numpy as np
from numpy import exp
import psrchive
import argparse
from astropy.coordinates import SkyCoord
import astropy.units as u

if "ipykernel_launcher.py" in sys.argv[0]:
    sys.argv = [sys.argv[0]]

parser = argparse.ArgumentParser(description='Code to create pulsar fits files')

parser.add_argument('-E', '--ephem', default='Cycle45Results/J1939+2134/BAND5/J1939+2134.txt',  help='Ephemeris file to update the model')
parser.add_argument('-Ms', '--mjd_start', type=float, default=58900.0, 
					help='Start MJD of the simulation span. (Def: 58900)')
parser.add_argument('-Mf', '--mjd_finish', type=float, default=59000.0,
					 help='End MJD  of the simulation span. (Def: 59000)')
parser.add_argument('-fl', '--freq_low', type=float, default=600.0,
					 help='Lowest frequency of the band. (Def: 250 MHz)')
parser.add_argument('-fh', '--freq_high', type=float, default=1400.0,
					 help='Highest frequency of the band. (Def: 500 MHz)')
parser.add_argument('-nc', '--nchan', type=int, default=128,
					 help='Number of channels. (Def: 128)')
parser.add_argument('-nb', '--nbin', type=int, default=1024,
					 help='Number of pulse phase bins. (Def: 1024)')
parser.add_argument('-ns', '--nsub', type=int, default=1,
					 help='Number of subintegrations. (Def: 10)')
parser.add_argument('-ts', '--tsub', type=float, default=10.0,
					 help='Subintegration time in secs. (Def: 10 secs)')
parser.add_argument('-N', '--nobs', type=int, default=10,
					 help='Total no. of observations between Ms ans Mf. (Def: 10)')
parser.add_argument('-np', '--npol', type=int, default=1,
					 help='Number of polarisations. Currently only works with 1 (Def: 1)')
parser.add_argument('-snr', '--snr', type=float, default=10,
					 help='Peak S/N ratio of each channel/subint (Def: 10)')
parser.add_argument('-noscat', '--noscat', action='store_true', default=False,
					 help='Do not scatter-broaden the pulsar (Def: False)')
parser.add_argument('-s', '--spidx', type=float, default=-1.7,
					 help='Spectral index of the pulsar (Def: -1.7)')
parser.add_argument('-fd', '--fluxd', type=float, default=10,
					 help='Flux density for scaling (Def: 1 mJy)')
parser.add_argument('-pbf', '--pbf', type=int, default=1,
					 help='PBF model for scattering (Def: 1).'+
					 		'1. Thin screen; 2. thick screen; '+
					 		'3. Truncated screen; 4. Continuous medium.')
parser.add_argument('-i', '--simpfile', type=str, default=None,
					 help='Simpulse input file (Def: PWD/simpulse_input.txt)')
parser.add_argument('-dmf', '--dmfile', type=str, default=None,
					 help='Input delta DM file (Def: None)')



def main():

	# Reads the terminal input arguments
	args = parser.parse_args()
	parfile = args.ephem
	if args.ephem == None:
		print ("\nError: Parameter file is not given. Exiting.\n")
		exit(0)

	if args.simpfile == None:
		simpfile = "simulation_results/result_simpulse_input/J1939_simpulse_input.txt"
	if args.simpfile != None:
		simpfile = args.simpfile
	if not os.path.isfile(str(simpfile)):
		print ("\nError: Input file is not given.")
		print("A sample file simpulse_input_J1939.txt is written out for reference.")
		print("Exiting.")
		write_samplefile()
		exit(0)
	nsub = args.nsub
	tsub = args.tsub
	npol = args.npol
	nchan = args.nchan
	nbin = args.nbin
	freq_low = args.freq_low
	freq_high = args.freq_high
	epoch_start = args.mjd_start
	epoch_stop = args.mjd_finish
	nobs = args.nobs
	snr = args.snr
	spidx = args.spidx
	fluxd = args.fluxd
	pbf = args.pbf
	if not args.noscat:
		if isinstance(pbf, int):
			if (pbf > 4):
				print ("Given PBF number %d doesn't exist! Exiting." % pbf)
				exit(0)
	# Reads the pulsar name from parameter file
	ar_psr, _, _, _ = readparfile(parfile)

	# Removes the simulation parameters out file, if already exists
	if os.path.isfile(ar_psr+"_dmalptscat.dm"):
		os.remove(ar_psr+"_dmalptscat.dm")

	# Creates DM timeseries or read from input file
	if args.dmfile == None:

		# Generates the epochs for simulation
		epochs = np.linspace(epoch_start, epoch_stop, nobs, endpoint=True, dtype=float)
		dDM = np.zeros(nobs)

		# Creates the DM timeseries
		dDM = make_dmseries(nobs, simpfile)

		# Creates tscat and alpha series
		tscat = np.zeros(nobs)
		alpha = np.zeros(nobs)

	else:
		# Reads epochs and delta DMs from the input file
		epochs = np.loadtxt(args.dmfile,usecols=0, dtype=float, comments='#',ndmin=1)
		dDM = np.loadtxt(args.dmfile,usecols=1, dtype=float, comments='#', ndmin=1)
		nobs = np.size(dDM)

		# Creates tscat and alpha series
		tscat = np.zeros(nobs)
		alpha = np.zeros(nobs)

	tscat, alpha = make_scatseries(dDM, simpfile)
	dmevents = make_dips(epochs, dDM, ar_psr, simpfile)
	dDM += dmevents
	# Runs the makefits() to generate simulated archives
	for obs in range(nobs):
		makefits(parfile, nsub, tsub, npol, nchan, nbin, freq_low, freq_high, snr, epochs[obs], dDM[obs], tscat[obs], alpha[obs], args.noscat, spidx, fluxd, pbf, simpfile)

########################################


def makefits(parfile, nsub, tsub, npol, nchan, nbin, freq_low, freq_high, snr, ar_mjd, dDM, tscat, alpha, noscat, spidx, fluxd, pbf, simpfile):
	"""
	Produces the simulated archive in psrfits format based on the input arguments.
	"""

	# Reads name, position and DM of the pulsar
	ar_psr, ar_raj, ar_decj, ref_dm= readparfile(parfile)
	ar_dm = float(ref_dm) + dDM

	# Creates the frequency array for sub-bands
	freq_c = freq_low + (freq_high - freq_low)/2
	bw = freq_high - freq_low
	chanwidth = bw / nchan
	freqArr = np.linspace(freq_low + (chanwidth/2.0), freq_low + bw - (chanwidth/2.0), nchan)
	tscat_cfr = tscat * (1000./freq_c)**(-1.*alpha)
	# Writes out the DM tscat and alpha values of the epoch
	f = open(ar_psr+"_dmalptscat.dm",'a')
	f.write("%.6f %.10f %.10f %.6f\n" % (ar_mjd, ar_dm, tscat, alpha))
	f.close()

	# Creates an instance of psrfits file. Only possible with ASP backend.
	ar = psrchive.Archive_new_Archive('ASP')
	# reshaping the data matrix
	ar.resize(nsub,npol,nchan,nbin)
	# Setting the DM of the epoch
	ar.set_dispersion_measure(float(ar_dm))
	# Setting source name
	ar.set_source(ar_psr)
	# Setting the pulsar position
	ar.set_coordinates(psrchive.sky_coord(ar_raj + ar_decj))
	# Setting the centre frequency of observation
	ar.set_centre_frequency(freq_c)
	# Setting the bandwidth of the archive
	ar.set_bandwidth(bw)
	# Setting the telescope. Resets ASP to GMRT.
	ar.set_telescope('DE601')

	# Setting MJD of the archive
	start_MJD = psrchive.MJD(ar_mjd)
	epoch = start_MJD+tsub/2.0

	# setting time of subints
	for subint in ar:
		subint.set_epoch(epoch)
		subint.set_duration(tsub)
		epoch += tsub
		# Setting the frequencies of the sub-bands
		for ichan in range(nchan):
			subint.set_centre_frequency(ichan,freqArr[ichan])

	# Setting the par file. It also creates the predictor table
	ar.set_ephemeris(parfile)
	# setting the archive as dedispersed
	ar.set_dedispersed(True)
	ar.dedisperse()

	# Getting the folding period fo converting the tscat to nbins
	p0 = ar.get_Integration(0).get_folding_period() * 1000.0

	# Getting the bandshape. Currently disabled as a better way is sought.
	#chanamps = bandshape(nchan)

	# Main loop that writes the data for each channel into the profiles of archive
	for isub in range(nsub):
		sub = ar.get_Integration(isub)

		for ipol in range(npol):

			for ichan in range(nchan):
				prof = sub.get_Profile(ipol,ichan)
				prof.get_amps()[:] = make_profile(nbin, snr, freqArr[ichan], ar_dm, tscat, alpha, p0, noscat, spidx, fluxd, pbf, ar_mjd, ar_psr, simpfile, chanwidth)
				#prof += chanamps[ichan] # Correcting for the bandshape

	# de-dedispersing the data and resetting the DM to reference DM.
	# This imitates the DM variation.
	ar.dededisperse()
	ar.set_dispersion_measure(float(ref_dm))

	# Writing the archive out and correcting for an important flag
	outfile = ar_psr+"."+str("%.3f"%ar_mjd)+"_"+str("%.0f" % freq_high)+"MHz."+ar.get_telescope()+".ar"
	ar.set_filename(outfile)
	ar.unload(outfile)
	os.popen("psredit -c aux:dmc=0 -m %s" % outfile).read()
	print ("Written out %s" % outfile)
	# Done.



def readparfile(parfile):

	"""
	Reads the input par file and collects pulsar name (PSRJ), right ascenscion (RAJ),
	Declination (DECJ), dispersion measure (DM). 
	"""

	ar_psr=None; ar_raj=None; ar_decj=None; ar_dm=0.; elong=0.0; elat=0.0

	with open(parfile,'r') as parf:

		for line in parf:
			line = line.strip()

			# Skip empty lines and comments
			if not line or line.startswith('#'):
				continue

			parts = line.split()

			if parts[0] == 'PSRJ':
				ar_psr = parts[1]

			if parts[0] == 'RAJ':
				ar_raj = parts[1]

			if parts[0] == 'DECJ':
				ar_decj = parts[1]

			if parts[0] == 'DM':
				ar_dm = parts[1]

	if ar_raj == None:
		with open(parfile,'r') as parf:
			for line in parf:
				line = line.strip()

				if not line or line.startswith('#'):
					continue

				parts = line.split()

				if parts[0] == 'ELONG':
					elong = float(parts[1])

				if parts[0] == 'ELAT':
					elat = float(parts[1])

		coo = SkyCoord(lon=elong*u.degree, lat=elat*u.degree,
				frame='geocentrictrueecliptic')
		coo_tmp = coo.transform_to("fk5")
		coo_conv = coo_tmp.to_string('hmsdms', sep=':')
		ar_raj = str(coo_conv.split(" ")[0])
		ar_decj = str(coo_conv.split(" ")[1])

	if (ar_psr == None or ar_raj == None or ar_decj == None):
		print ("\nError: Pulsar name and/or position are not given. Exiting.\n")
		exit(0)

	return(ar_psr, ar_raj, ar_decj, ar_dm)



def bandshape(nchan):

	"""
	A stupid way to create a bandshape. Could be improved 
	to have a better bandshape so that the S/N across the 
	band resembles that.
	"""

	sqband = np.zeros(nchan, dtype=float)

	for i in range (nchan):

		if (i > 0.02*nchan ):
			sqband[i] = 1

	chans = np.linspace(0., nchan, nchan, dtype=int)
	chans = chans.astype(float)
	phase = nchan/2.
	sigma = nchan*0.1
	gauss = exp(-chans/15.)
	gauss = np.flip(gauss)
	bandshape = convolve_funcs(gauss,sqband)
	bandshape += np.random.normal(0,0.05,nchan)
	bandshape /= np.median(bandshape)
	bandshape = (10**bandshape)*10.

	return(bandshape)


def convolve_funcs(func1, func2):

	"""
	Algorithm implementing circular convolution of two functions.
	Currently using the FFT based convolution as it is way faster
	than the usual way.
	"""

	# Checking the size of the arrays and zero padding if one of 
	# them is shorter than the other.
	n1 = np.size(func1)
	n2 = np.size(func2)

	if n2 < n1:
		zeros = np.zeros(n1-n2).tolist()
		func2 = func2.tolist() + zeros
		func2 = np.asarray(func2, dtype=float)

	if n1 < n2:
		zeros = np.zeros(n2-n1).tolist()
		func1 = func1.tolist() + zeros
		func1 = np.asarray(func1, dtype=float)

	# Taking the fft of both the functions
	f1_fft = np.fft.fft(func1)
	f2_fft = np.fft.fft(func2)

	# Multiplying the fft of two functions
	m = f1_fft * f2_fft

	# Gives out the final convolved function
	y = abs(np.fft.ifft(m))

	return(y)


# Creates a power law spectrum.
def generate_spectrum(freqs, amp, beta):

	return amp * freqs ** beta

# Creates a timeseries from the power spectrum.
def make_timeseries(amp, beta, ndays):

	"""
	Creates a timeseries using the given spectral index (beta) and amplitude
	(amp). Random noise is also added to the spectrum.
	"""
	nfreqs = 10000
	freqs = np.linspace(1e-3,2,nfreqs)
	powersp   = generate_spectrum(freqs,amp,beta) * np.random.rand(len(freqs))
	fouriersp = np.sqrt(powersp) * np.exp(1j*np.pi*np.random.rand(len(powersp)))
	Tseriesfft = np.concatenate(([0],fouriersp,np.conj(fouriersp[::-nfreqs])))
	Tseries = np.real(np.fft.ifft(Tseriesfft))[1:nfreqs+1]
	Tseries = Tseries[:-100]; Tseries = Tseries[100:]
	freqs = freqs[:-100]; freqs = freqs[100:]

	phase_points = np.linspace(1e-3,1,ndays)
	Tseries = np.interp(phase_points, freqs, Tseries)
	return (Tseries)


# Creates DM series based on the input
def make_dmseries(n, simpfile):
	"""
	Creates a DM time series based on the amplitude and power given in dmsp
	line of the input file.
	"""
	amp = 1e-6; power = -2.0
	for line in open(simpfile):
		if line.startswith("dmsp"):
			data = line.split()
			try: 
				amp = float(data[1])
			except:
				print ("Amplitude is not given. Using 1e-6.")
			try: 
				power = float(data[2])
			except:
				print ("Spectral index is not given. Using -2.0")

	dmseries = np.zeros(n)

	dmseries = make_timeseries(amp, power, n)

	return (dmseries)


def make_dips(epochs, dmseries, psr, simpfile):
	epoch_dip=[]; amplitude=[]; rtime = [];param=[]
	for line in open(simpfile):
		if line.startswith("edip"):
			data = line.split()
			epoch_dip.append(float(data[1]))
			amplitude.append(float(data[2]))
			rtime.append(float(data[3]))
			param.append(str(data[4]))

	if not (len(epoch_dip) == 0):
		x = np.linspace(0,(epochs[-1]-epochs[0]), np.size(epochs),endpoint=True, dtype=float)
		tada = np.absolute(epochs-epoch_dip)
		dipx = tada.argmin()
		dip = np.zeros(np.size(epochs))
		if "gauss" in param[0]:
			dip = np.roll(amplitude*((1.-np.exp(-x/rtime))), dipx)
			f = open(psr+"_amplitudes.txt", 'w')
			for i in range(np.size(epochs)):
				f.write("%f %f\n" % (epochs[i],dip[i]-amplitude))
			f.close()
			return (np.zeros(np.size(epochs)))
		if param[0] == "dm":
			dip = np.roll(amplitude*((1.-np.exp(-x/rtime))), dipx)
			return(dip-amplitude)
	if (len(epoch_dip) == 0):
		return(np.zeros(np.size(epochs)))

# Generates the scattering timeseris.
def make_scatseries(dmseries, simpfile):

	"""
	Creates a scattering time series based on the arguments from the 
	input file. The line should be in the format 
	"scat <tscat> <alpha> <rms> <mode> <param>". The value of tscat
	should be at 1 GHz reference. The value of alpha should be negative.
	mode can be 
	"stable" - same alpha/tscat for all epochs; 
	"dm" - same variation as DM timeseries in alpha/tscat;
	"random" - vary alpha/tscat randomly with the given rms 
	"powerlaw" - Use the given alpha to create a tscat series
	param can be "tscat", "alpha" or "none". Mode powerlaw will only 
	work with "tscat". The variables are read as positional arguments, 
	so be careful with that.

	"""

	# Reads the different scattering variables from the input file.
	tsc_ref = []; alp_ref = []; param_rms = []; mode=[]; param_ref = []

	for line in open(simpfile):

		if line.startswith("scat"):
			data = line.split()
			tsc_ref.append(float(data[1]))
			alp_ref.append(float(data[2]))
			param_rms.append(float(data[3]))
			mode.append(data[4])
			param_ref.append(data[5])
	tsc_ref = np.asarray(tsc_ref,dtype=float)
	alp_ref = np.asarray(alp_ref,dtype=float)
	param_rms = np.asarray(param_rms,dtype=float)

	# Creates a timeseries in tscat and alpha same as dm variation.
	# Only one of those can be varied w.r.t dm, not both.
	if (mode[0] == "dm"):
		medDM = np.median(dmseries)

		if (param_ref[0] == "tscat"):
			tt = dmseries / np.sum(dmseries)
			tt = tt* (np.std(tt) * (1./np.max(tt)))
			tscat = tsc_ref + (tsc_ref * tt/10.)
			tscat = tsc_ref + (dmseries / medDM)/100.
			tscat += np.min(tscat)*-1.
			alpha = alp_ref * np.ones(np.size(dmseries), dtype=float)

		if (param_ref[0] == "alpha"):
			tt = dmseries / np.sum(dmseries)
			tt = tt* (np.std(tt) * (1./np.max(tt)))
			alpha = alp_ref + (alp_ref * tt/100.)
			tscat = tsc_ref * np.ones(np.size(dmseries), dtype=float)

		return (tscat, alpha)

	# Creates a timeseries of tscat and alpha with the same input values.
	if (mode[0] == "stable"):
		tscat = tsc_ref * np.ones(np.size(dmseries), dtype=float)
		alpha = alp_ref * np.ones(np.size(dmseries), dtype=float)

		return (tscat, alpha)

	# Creates a tscat or dm variable timeseries with a given rms
	if (mode[0] == "random"):

		if (param_ref[0] == "none" and param_ref != "tscat" and param_ref !="alpha"):
			print ("\nERROR: Wrong combination of mode \"%s\" with param \"%s\"."%(mode[0],param_ref[0]))
			print("Change the variables accordingly to re-run simulator.\n")
			exit(0)

		if (param_ref[0] == "tscat"):

			if ((tsc_ref - param_rms * 5.) < 0.):
				param_rms = tsc_ref / 5.
				print("Warning: RMS tscat makes it go negative. Changing it to %f"%(param_rms))
			tscat = np.random.normal(tsc_ref, param_rms, np.size(dmseries))
			alpha = alp_ref * np.ones(np.size(dmseries), dtype=float)

			return (tscat, alpha)

		if (param_ref[0] == "alpha"):
			tscat = tsc_ref * np.ones(np.size(dmseries), dtype=float)
			alpha = np.random.normal(alp_ref, param_rms, np.size(dmseries))
			#alpha = alp_ref * np.random.rand(alp_ref, param_rms, np.size(dmseries))

			return (tscat, alpha)

	# Creates a tscat timeseries based on the powerlaw
	if (mode[0] == "powerlaw"):
		tscat = make_timeseries(1e-8, alp_ref, np.size(dmseries))
		tscat += np.min(tscat)*-1
		tscat /= 10.
		tscat += tsc_ref
		alpha = alp_ref * np.ones(np.size(dmseries), dtype=float)

		return(tscat, alpha)


# Creates a Gaussian shaped model
def make_gaussian(x, amp, phase, sigma):
	gauss = amp * exp( -(x-phase)**2./(2.*sigma**2.) )
	return ( gauss )

# Creates a Lorentzian shaped profile
def make_lorentzian( x, x0, a, gam ):
	lorentzian = a * gam**2 / ( gam**2 + ( x - x0 )**2)
	return(lorentzian)
# Thin scattering screen approximation
def pbf1(x, tscat):
	return ( exp(-x/tscat) )

# Thick scattering screen approximation (Eq. 13 in Williamson 1972)
def pbf2(x, tscat):
	x += 1e-20
	amp = np.sqrt( (np.pi * tscat) /(4 * x**3) )
	expo = (np.pi**2 * tscat) / (16 * x)
	return ( amp * exp(-expo) )

# Truncated screen approximation (Eq. 3 in Geyer et al. 2017)
def pbf3(x, tscat):
	x += 1e-20
	amp = 1./np.sqrt(np.pi * tscat * x)
	# suboptimal. But a workaround to avoid the zero bin peak
	amp[0] = amp[1]+amp[2]
	return ( amp * ( exp(-x/tscat) ) )

# Continuous media approximation (Eq 21 in Williamson 1972)
def pbf4(x, tscat):
	x += 1e-20
	amp = np.sqrt( (np.pi**5 * tscat**3) / (8*x**5) )
	expo = (np.pi**2 * tscat) / (4.*x)
	return( amp * exp(-expo) )


# Amount of Dispersion Measure smearing 
def tau_DM(freq_sc, chanwidth, dm):
    freq0 = freq_sc/1000   #central frequency in GHz
    B = chanwidth # MHz
    
    tau_dm_us = 8.3 * B * dm * freq0**(-3)
    tau_dm_ms = tau_dm_us * 1e-3
    
    n_dm = tau_dm_ms / p0 * nbin
    return tau_dm_ms

def apply_dm_smearing(profile, dm, freq_MHz, chanwidth_MHz, period_s):
    """
    Apply symmetric boxcar DM smearing without shifting peak.
    Works for frequency-scrunched single profile.

    profile        : 1D numpy array
    dm             : dispersion measure (pc cm^-3)
    freq_MHz       : observing frequency (MHz)
    chanwidth_MHz  : channel width (MHz)
    period_s       : pulsar period (seconds)
    """

    nbin = len(profile)

    # ---- DM smearing time in seconds ----
    # 8.3e6 gives microseconds
    delta_t_us = 8.3e6 * dm * chanwidth_MHz / (freq_MHz**3)

    delta_t_s = delta_t_us * 1e-6

    # ---- Convert to bins ----
    smear_bins = delta_t_s / period_s * nbin

    smear_bins = int(np.round(smear_bins))

    if smear_bins < 0.5:
        return profile

    # ---- Make symmetric boxcar ----
    kernel = np.ones(smear_bins)
    kernel /= kernel.sum()

    # ---- Convolve WITHOUT changing length ----
    smeared = np.convolve(profile, kernel, mode='same')

    return smeared


# Generates the pulse profile
def make_profile(nbin, snr, freq, dm, tscat, alpha, p0, noscat, spidx, fluxd, pbf, mjd, psr, simpfile, chanwidth):

	"""
	Generates the profile based on the input params. One can define
	as many gaussian components as required in the file. The profile
	also gets scatter-broadened, if asked.
	"""

	x = np.arange(nbin, dtype=float)
	g_amp = []; g_phase = []; g_sigma = []
	g_wsc = []; g_hsc = []; g_psc = []
	l_amp = []; l_phase = []; l_gamma = []
	l_gsc = []; l_hsc = []; l_psc = []

	prof = np.zeros(nbin)
	dip_condition = False; param_dip = []

	# Reads the data of profile gaussian components from the file
	for line in open(simpfile):

		if line.startswith("gauss"):
			data = line.split()
			g_phase.append(float(data[1]) * nbin)
			g_amp.append(float(data[2]))
			g_sigma.append((float(data[3]) / 2.35482) * nbin)
			g_psc.append(float(data[4]))
			g_hsc.append(float(data[5]))
			g_wsc.append(float(data[6]))

		if line.startswith("lorentz"):
			data = line.split()
			l_phase.append(float(data[1]) * nbin)
			l_amp.append(float(data[2]))
			l_gamma.append(float(data[3]) * nbin)
			l_psc.append(float(data[4]))
			l_hsc.append(float(data[5]))
			l_gsc.append(float(data[6]))

		if line.startswith("edip"):
			dip_condition = True
			data = line.split()
			param_dip.append(str(data[4]))
			param_dip = param_dip[0]

	# Makes the combined profile
	for i in range(np.size(g_amp)):

		gstr = 'gauss' + str(i + 1)

		if dip_condition == True:
			if param_dip.startswith('gauss'):

				a_f = 0
				dd = np.loadtxt(psr + "_amplitudes.txt", dtype=float, usecols=0)
				ap = np.loadtxt(psr + "_amplitudes.txt", dtype=float, usecols=1)
				xp = np.where(np.isclose(dd, mjd))[0][0]

				p_f = g_phase[i] * (freq / 1000.) ** g_psc[i]
				w_f = g_sigma[i] * (freq / 1000.) ** g_wsc[i]

				if gstr == param_dip:
					if np.size(ap) > 1:
						a_f = (ap[xp] + g_amp[i]) * (1.0 / freq) ** g_hsc[i]
					if np.size(ap) == 1:
						a_f = (g_amp[i] + ap) * (1.0 / freq) ** g_hsc[i]

				if gstr != param_dip:
					a_f = g_amp[i] * (1.0 / freq) ** g_hsc[i]

				prof += make_gaussian(x, a_f, p_f, w_f)

		if (dip_condition == False) or (param_dip == 'dm'):
			p_f = g_phase[i] * (freq / 1000.) ** g_psc[i]
			w_f = g_sigma[i] * (freq / 1000.) ** g_wsc[i]
			a_f = g_amp[i] * (1.0 / freq) ** g_hsc[i]
			prof += make_gaussian(x, a_f, p_f, w_f)

	for i in range(np.size(l_amp)):
		p_f = l_phase[i] * (freq / 1000.) ** l_psc[i]
		w_f = l_gamma[i] * (freq / 1000.) ** l_gsc[i]
		a_f = l_amp[i] * (1.0 / freq) ** l_hsc[i]
		prof += make_gaussian(x, a_f, p_f, w_f)

	# Scale flux
	prof /= np.sum(prof)
	fscale = fluxd * ((freq / 1400) ** spidx)
	prof *= fscale

	# Scatter broadening
	if not noscat:

		tscat_f = tscat * ((freq / 1000.) ** alpha)
		tscat_bins = nbin * (tscat_f / p0)

		scmodel = ("pbf%d" % pbf)
		scatfunc = globals()[scmodel]
		scatprof = scatfunc(x=x, tscat=tscat_bins)

		prof = convolve_funcs(prof, scatprof)

	# DM smearing (apply after scattering)
	prof = apply_dm_smearing(prof, dm, freq, chanwidth, p0)


	# --------------------------
	# Dispersion smearing
	# --------------------------

	'''nu = freq / 1000.0
	B = chanwidth

	tau_dm_us = 8.3 * B * dm * nu ** (-3)
	tau_dm_ms = tau_dm_us * 1e-3

	n_dm = tau_dm_ms / p0 * nbin
	k = int(np.round(min(n_dm, nbin - 1)))

	if k > 1:
		kernel = np.ones(k) / k
		prof = convolve_funcs(prof, kernel)'''

	# Noise
	noise = np.random.normal(0, np.max(prof) / snr, len(prof))

	return prof + noise

def write_samplefile():
	f = open("simpulse_input.txt",'a')
	f.write("# all the values are referenced to 1 GHz\n")
	f.write("# gauss_no. / phase(0-1) / amplitude / w50(0-0.5) / Pscaling / Hscaling / Wscaling\n")
	f.write("gauss1 0.5 1.0 0.05 0.0 0.0 0.0\n")
	f.write("# lorentz_no. / phase (0-1) / amplitude / gamma (width) / Xscaling / Ascaling / Gscaling\n")
	f.write("#lorentz1 0.1 1.0 0.03 0.0 0.0 0.0\n\n")
	f.write("# scattering time reference\n")
	f.write("# scat_time (ms) / alpha / rms / mode / param\n")
	f.write("scat 0.005 -4.4 0.4 stable tscat\n")
	f.write("# mode can be \"stable\" - same alpha/tscat for all epochs; \"dm\" - same variation as DM timeseries in alpha/tscat;\n")
	f.write("# \"random\" - vary alpha/tscat randomly with the given rms; and \"powerlaw\" - Uses the given alpha to create a tscat series\n")
	f.write("# param can be \"tscat\", \"alpha\" or \"none\". Mode powerlaw will only work with \"tscat\"\n\n")
	f.write("# DM spectral model\n")
	f.write("#     amplitude   spectral_index\n")
	f.write("dmsp    1e-6        -2.0\n")
	f.close()

main()

Written out J1939+2134.58900.000_1400MHz.DE601.ar
Written out J1939+2134.58911.111_1400MHz.DE601.ar
Written out J1939+2134.58922.222_1400MHz.DE601.ar
Written out J1939+2134.58933.333_1400MHz.DE601.ar
Written out J1939+2134.58944.444_1400MHz.DE601.ar
Written out J1939+2134.58955.556_1400MHz.DE601.ar
Written out J1939+2134.58966.667_1400MHz.DE601.ar
Written out J1939+2134.58977.778_1400MHz.DE601.ar
Written out J1939+2134.58988.889_1400MHz.DE601.ar
Written out J1939+2134.59000.000_1400MHz.DE601.ar


In [28]:
!psrplot -p freq+ J1939+2134.59000.000_1400MHz.DE601.D -D /XS

In [27]:
!pam -D -e D J1939+2134.59000.000_1400MHz.DE601.ar
!pam -F -e FD J1939+2134.59000.000_1400MHz.DE601.ar J1939+2134.59000.000_1400MHz.DE601.D
!psrplot -p flux J1939+2134.59000.000_1400MHz.DE601.FD -D /XS

J1939+2134.59000.000_1400MHz.DE601.D written to disk
J1939+2134.59000.000_1400MHz.DE601.FD written to disk
J1939+2134.59000.000_1400MHz.DE601.FD written to disk


In [ ]:
!pav -F -e F J0613-0200.59000.000_1450MHz.DE601.ar J0613-0200.59000.000_1450MHz.DE601.F -D/XS

 Type <RETURN> for next page: 

In [72]:
!pdv -FTt J0740+6620.59000.000_1450MHz.DE601.FD > J0740+6620_sim.txt
!head J0740+6620_sim.txt

File: J0740+6620.59000.000_1450MHz.DE601.FD Src: J0740+6620 Nsub: 1 Nch: 1 Npol: 1 Nbin: 1024 RMS: 0.000117828
0 0 0 0.00299393
0 0 1 0.00329625
0 0 2 0.00350525
0 0 3 0.00380163
0 0 4 0.0039294
0 0 5 0.00418578
0 0 6 0.00427992
0 0 7 0.00452071
0 0 8 0.00498178


In [9]:
!pdv -h

pdv: option requires an argument -- 'h'
A program for extracting archive data in text form 
Usage: 
     pdv [options] filenames 
     pdv help [arg] 
Where: 
   -b ibin     select a single phase bin, from 0 to nbin-1 
   -n ichan    select a single frequency channel, from 0 to nchan-1 
   -i isub     select a single integration, from 0 to nsubint-1 
   -r phase    rotate the profiles by phase before printing 
   -F          Fscrunch first 
   -T          Tscrunch first 
   -p          Pscrunch first 
   -C          Centre first 
   -B factor   Bscrunch by this factor first 
   -j job      Do anything first 
   -J script   Do many things first 
   -x          Convert to Stokes, print fraction polarisation (L & |V| biased corrected) 
   -l          Convert to Stokes, print fraction linear, biased corrected (only with -t) 
   -Z          Convert to Stokes and also print position angle (P.A.) 
   -z          Convert to Stokes and also print ellipticity angle 
   -L sigma    Minimum linear

In [61]:
!pam -F -T -e FT Cycle45Results/J0740+6620/BAND5/J0740+6620_modified.fits
!pdv -FTt Cycle45Results/J0740+6620/BAND5/J0740+6620_modified.fits > Cycle45Results/J0740+6620/BAND5/J0740+6620_data.txt
!head Cycle45Results/J0740+6620/BAND5/J0740+6620_data.txt

Cycle45Results/J0740+6620/BAND5/J0740+6620_modified.FT written to disk
File: Cycle45Results/J0740+6620/BAND5/J0740+6620_modified.fits Src: J0740+6620 Nsub: 1 Nch: 1 Npol: 1 Nbin: 64 RMS: 0.00687016
0 0 0 0.0466435
0 0 1 0.0380731
0 0 2 0.0785926
0 0 3 0.102768
0 0 4 0.107456
0 0 5 0.0425309
0 0 6 0.0084243
0 0 7 0.0129233
0 0 8 0.00132974


TypeError: expected x and y to have same length

In [11]:
freq = 1300
bw = 1450-1150
nchan = 128
chanwidth = bw/nchan
dm=71.0151

nu = freq / 1000.0
B = chanwidth

tau_dm_us = 8.3 * B * dm * nu ** (-3)
tau_dm_ms = tau_dm_us * 1e-3

print('DM smear time in ms :', tau_dm_ms)

DM smear time in ms : 0.6287963664940828


In [29]:
nu1 = 1.150 #GHz
nu2 = 1.450 #GHz
dm = float(input("give dm of the pulsar in ms: "))
delta_t = 4.15 * dm * (nu1**(-2) - nu2**(-2))
print("time delay in ms:", delta_t)
po = float(input("period of the pulsar in ms:"))
print(delta_t/po)

give dm of the pulsar in ms:  9.00


time delay in ms: 10.477399980669338


period of the pulsar in ms: 16.05


0.6527975065837593
